# Introduction to flux analysis at a genome-scale using Cobrapy
In this tutorial, you'll learn how to:
1. Load and explore metabolic models
2. Run Flux Balance Analysis (FBA)
3. Perform Flux Variability Analysis (FVA)
4. Add custom constraints (like flux ratios)
5. Compare conditions and visualize flux changes

A Cobrapy documentation can be found here:
https://cobrapy.readthedocs.io/en/latest/

Contact: Dr. Anika Küken (ankueken@uni-potsdam.de), Bioinformatics Department, University of Potsdam

## Part 1: Load and inspect a model 

In this part we are gonna use a genome-scale metabolic model of Arabidopsis thaliana (AraCore).
Store the model in your current working folder.

Further information you can find from its original publication: \
https://academic.oup.com/plphys/article/165/3/1380/6113226?login=true

In [ ]:
import cobra
from cobra.io import read_sbml_model

# Load the Arabidopsis core model (AraCore.xml)
model = read_sbml_model('AraCore.xml')

A model object usually has the following attributes that you can asses by model.attr_name 

| **Attribute**   | **Type**                     | **Description** |
|-----------------|------------------------------|-----------------|
| `id`            | `str`                        | Model ID |
| `name`          | `str`                        | Human-readable name |
| `reactions`     | `ReactionList`               | All reactions in the model |
| `metabolites`   | `MetaboliteList`             | All metabolites in the model |
| `genes`         | `GeneList`                   | All associated genes |
| `objective`     | `cobra.Objective`            | Objective function (e.g., biomass reaction) |
| `compartments`  | `dict`                       | Mapping of compartment IDs to names |
| `notes`         | `dict`                       | Free-form notes and metadata |
| `annotation`    | `dict`                       | External database identifiers (KEGG, BiGG, etc.) |


### Inspecting the model numbers

In [ ]:
# 💡 Explanation: Add your code comments below for clarity.
from cobra.util.array import create_stoichiometric_matrix

S = create_stoichiometric_matrix(model)

### Task 1.1: How many metabolites, reactions and genes are contained in the model?

Inspect the number of rows and columns in the stoichiometric matrix and compare to size of model attributes
- reactions
- metabolites
- genes

In [ ]:
# TODO
# add your code here


In [ ]:
# Where can we find information about gene reaction association?
model.reactions.RBC_h.gpr

### Reaction boundaries

### Reversible reactions in stoichiometric models

              A  B
A -> B: S = [-1  1]'  

B -> A: S = [ 1 -1]'

In [ ]:
# Inspecting the model exchange fluxes
model.boundary
[reaction.bounds for reaction in model.boundary]

In [ ]:
model.boundary[1]

In [ ]:
# 💡 Explanation: Add your code comments below for clarity.
model.boundary[len(model.boundary)-1]

### Model objective

The model has different biomass reactions:

 - Bio_opt   growth under light limiting conditions
 - Bio_NLim  growth under nitrogen limiting conditions
 - Bio_CLim  growth under carbon limiting conditions

In [ ]:
# Check the objective function and find the reaction optimized
print(model.objective)

print(model.reactions.Bio_opt.objective_coefficient)
for r in model.reactions:
    if r.objective_coefficient != 0:
        print(r.id)
        print(r.name)

### Task 1.2: Biomass reactions and compartments
Have a look at the different biomass reactions listed above

In [ ]:
# TODO
# add your code here


What is the compartment of reaction RBC_h? \
Which other compartments are considered in the model?

In [ ]:
# TODO 
# add your code here


## Part 2: Flux analysis and integration of additional constraints

The current objective is to optimize biomass reactio Bio_opt. To run a flux balance analysis (FBA) and find the optimal specific growth rate you can use:

    solution = model.optimize()

In [ ]:
solution = model.optimize()

print("Objective value: %.3f\n" % solution.objective_value)
print("Status: %s\n" % solution.status)

print("Fluxes:\n")
print(solution.fluxes)

The solution obtained from FBA are usually not unique. To get flux ranges we can run flux variability analysis (FVA). 

In [ ]:
# run FVA, per default the range is calculated at optimum biomass, we can allow deviation by specifying
# the percentage of optimal biomass considered (we use 99%) 

solution_fva_BioOpt = cobra.flux_analysis.flux_variability_analysis(model,fraction_of_optimum=0.99)
print(solution_fva_BioOpt)

### Task 2.1: Inspect flux solutions

Inspect the reactions 'Im_hnu', 'Im_CO2', 'Im_NO3', 'Im_NH4', 'RBC_h', 'RBO_h' 

- What are the reaction names that belong to the reaction id's given above? 
- What is the flux value for the reactions above obtained from FBA?
- What is the flux range at optimal growth considering objective Bio_Opt?

Given the values from FBA. What is the ratio of Rubisco carboxylation to oxygenation?

In [ ]:
print('Im_CO2')
print("FBA value: %.3f" % solution.fluxes['Im_CO2'])
print("FVA min value: %.3f" % solution_fva_BioOpt.minimum['Im_CO2'])
print("FVA max value: %.3f\n" % solution_fva_BioOpt.maximum['Im_CO2'])

# TODO
# add your code here



### Adding ratio constraints for improved predicitions

In [ ]:
# How can we improve this?

# We can add constraints restricting the ratio to physiological ratios
# 1.5 <= RBC_h/RBO_h <= 4

# constraint 1: 0 <= RBC_h - 1.5*RBO_h
# constraint 2: RBC_h - 4*RBO_h <= 0
c1 = model.problem.Constraint(model.reactions.RBC_h.flux_expression - 1.5 * model.reactions.RBO_h.flux_expression, lb=0)  
c2 = model.problem.Constraint(model.reactions.RBC_h.flux_expression - 4 * model.reactions.RBO_h.flux_expression, ub=0) 

model.add_cons_vars([c1, c2])

### Task 2.2: Flux analysis under new constraints

What is the maximal growth rate when the additional constraints are considered?
Calculate flux ranges considering at least 99% of the optimum biomass flux. 


In [ ]:
# TODO
# add your code here


### Task 2.3: What is the prediced storage rate of starch (reaction id: Ex_starch)?

In [ ]:
model.reactions.Ex_starch

In [ ]:
# TODO
# add your code here


In [ ]:
# set the flux of starch storage to at least 0.2 mmol/gDW/h and rerun FBA and FVA
model.reactions.Ex_starch.lower_bound = 0.2

solution_starch_export_on = model.optimize()
solution_fva_starch_export_on = cobra.flux_analysis.flux_variability_analysis(model,fraction_of_optimum=0.99)
solution_starch_export_on.fluxes['Ex_starch']


### Task 2.4: Flux comparison

- Compare the biomass predictions obtained from FBA.
- Compare the flux ranges. To this end we want to identify reactions whose flux ranges do not overlap in the two scenarios. 
- Find the reactions with the largest changes.

In [ ]:
import pandas as pd

fva_df = pd.DataFrame({
    "min1": solution_fva_co_ratio_constr["minimum"],
    "max1": solution_fva_co_ratio_constr["maximum"],
    "min2": solution_fva_starch_export_on["minimum"],
    "max2": solution_fva_starch_export_on["maximum"]
})

# TODO
# add your code here

